In [24]:
from torch_geometric.data import Data
import random
from utils import *
import torch
import torch.nn.functional as F

In [27]:
import torch.nn as nn
from torch_geometric.nn import GATConv, GCNConv
class GAT(nn.Module):
    def __init__(self,
                 g,
                 num_layers,
                 in_dim,
                 num_hidden,
                 heads,
                 attn_drop,
                 negative_slope):
        super(GAT, self).__init__()
        self.g = g
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        # self.gat_layers = nn.ModuleList()
        self.gat1 = GATConv(in_dim, num_hidden, heads, dropout= attn_drop, add_self_loops  = False, negative_slope = negative_slope)
        # self.gat_layers.append(GATConv(in_dim, num_hidden, heads, dropout= attn_drop, add_self_loops  = False, negative_slope = negative_slope))
        # hidden layers
        # for l in range(1, num_layers):
        #     # due to multi-head, the in_dim = num_hidden * num_heads
        #     self.gat_layers.append(GATConv(in_dim, num_hidden, heads, dropout= attn_drop, add_self_loops  = False, negative_slope = negative_slope))

    def forward(self, inputs):
        heads = []
        h = inputs
        # get hidden_representation
        for l in range(self.num_layers):
            h =self.gat_layers[l](self.g, h)
        # get heads
        for i in range(h.shape[1]):
            heads.append(h[:, i])
        return heads

In [ ]:
features = torch.randn(50, 5)
graph_data = pickle_load("/workspace/Synthetic_graph/Training_graph_50.pkl")
g = graph_data[0]
x = torch.eye(g.number_of_nodes())  # 节点特征
edge_index = torch.tensor(list(g.edges)).t().contiguous()
data = Data(x=x, edge_index=edge_index)
model = GAT(data,num_layers = 1, in_dim = 5, num_hidden = 32, heads = 4, attn_drop = 0.5, negative_slope = 0.2)
model.train()
output = model(features)

In [ ]:
class LearnModify(nn.Module):
    def __init__(self, num_features, graph_embedding_size, epoch = 100, lr = 0.0001, modified_edge = 100, GraphNumber = 50, Graphsize = 50, num_heads = 4):
        super(LearnModify, self).__init__()
        self.epoch = epoch
        self.lr = lr
        self.modified_edge = modified_edge
        self.GraphNumber = GraphNumber
        self.Graphsize = Graphsize
        self.GAT_num_hidden = 32
        self.GAT_num_layers = 1
        self.GAT_num_features = num_features
        self.GAT_attn_drop = 0.5
        self.negative_slope = 0.2
        self.num_heads = num_heads
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_layers = 1
num_feats = 50
num_hidden = 32
in_drop = 0.6
attn_drop = 0.5
negative_slope = 0.2
heads = 4
graph_data = pickle_load("/workspace/Synthetic_graph/Training_graph_50.pkl")
g = graph_data[0]
model = GAT(g,
            num_layers,
            num_feats,
            num_hidden,
            heads,
            F.elu,
            in_drop,
            attn_drop,
            negative_slope)